In [ ]:
# installation of Weights & Biases
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.6/248.6 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.3 MB/s eta 0:00:00


In [ ]:
# Importing libraries
import logging
import tempfile
import pandas as pd
import os
import wandb
from sklearn.model_selection import train_test_split

In [ ]:
# wandb login
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


##1.3 Data segregation


In [ ]:
# variables of the code

# proportion used to separate the dataset between training and testing
# we chose the proportion 80% training and 20% testing, using a simple random division
# This was the simplest and most recommended way for an initial division
test_size = 0.2

# seed used for the randomness of the splint to be controlled and reused
seed = 41

# reference (column) to stratify the data
# separates maintaining an equal proportion of column values
stratify = "class"

# name of the artifact that we will get from wandb and divide between training and testing
artifact_input_name = "flamigos/Murshroom-Kmeans/clean_data_dataset:latest"

# artifact type
artifact_type = "segregated_data"

In [ ]:
# configure logging
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(message)s",
                    datefmt='%d-%m-%Y %H:%M:%S')

# reference to a logger object
logger = logging.getLogger()

# initializing a new wandb run
run = wandb.init(entity="flamigos", project="Murshroom-Kmeans", job_type="split_data")

logger.info("Downloading and reading artifact")
artifact = run.use_artifact(artifact_input_name)
artifact_path = artifact.file()
df = pd.read_csv(artifact_path)


logger.info("Splitting data into train and test")
splits = {}

# dividing the dataset between training and testing and storing it in the splits dictionary
splits["train"], splits["test"] = train_test_split(df,
                                                   test_size=test_size,
                                                   random_state=seed,
                                                   stratify=df[stratify])

wandb: Currently logged in as: francisco-valmir (flamigos). Use `wandb login --relogin` to force relogin


In [ ]:
# to save the artifacts. We use a temporary directory to leave no traces
with tempfile.TemporaryDirectory() as tmp_dir:

    # does the entire operation of saving the 2 artifacts in wandb
    for split, df in splits.items():

        # creates the artifact name through the dictionary value
        artifact_name = f"{split}.csv"

        # Gets the location of the temporary volume created
        temp_path = os.path.join(tmp_dir, artifact_name)

        logger.info(f"Uploading the {split} dataset to {artifact_name}")

        # save to wandb
        df.to_csv(temp_path,index=False)

        artifact = wandb.Artifact(name=artifact_name,
                                  type=artifact_type,
                                  description=f"{split} split of dataset {artifact_input_name}",
        )
        artifact.add_file(temp_path)

        logger.info("Logging artifact")
        run.log_artifact(artifact)

        # This waits for the artifact to be loaded into W&B. If you don't add this,
        # the temporary directory may be removed before W&B has a chance to do so.
        # uploading the datasets, and the upload may fail
        artifact.wait()

In [ ]:
# closes wandb run
run.finish()